In [1]:
using Dojo
using DojoEnvironments
using LinearAlgebra

# Environment
quadrotor_env = get_environment(:quadrotor_waypoint; horizon=1000)

# Constants for the PD controllers
kp_thrust = 10.0
kd_thrust = 5.0
kp_yaw = 1.0
kd_yaw = 0.1
kp_pitch = 1.0
kd_pitch = 0.1
kp_roll = 1.0
kd_roll = 0.1

# Motor mixing algorithm
function motor_mixing(thrust, yaw, pitch, roll)
    motor1 = thrust + yaw + pitch + roll
    motor2 = thrust - yaw + pitch - roll
    motor3 = thrust - yaw - pitch + roll
    motor4 = thrust + yaw - pitch - roll
    return [motor1, motor2, motor3, motor4]
end

# PD controllers
function pd_control(error, error_dot, kp, kd)
    return kp * error + kd * error_dot
end

# Thrust and orientation controller
function control_thrust_orientation!(environment, v_des)
    state = get_state(environment)
    position = state[1:3]
    println("Position: ", position)
    orientation = state[4:6]  # Current orientation in axis-angle form
    linear_velocity = state[7:9]
    angular_velocity = state[10:12]

    # Calculate thrust
    error_v = v_des .- linear_velocity
    thrust = pd_control(error_v, -linear_velocity, kp_thrust, kd_thrust)
    thrust_total = norm(thrust) + 30.0  # Adding a higher constant value to ensure lift-off

    # Orientation control
    pitch_des = atan(v_des[2], v_des[3])  # Desired pitch angle to achieve desired velocity in y
    roll_des = atan(v_des[1], v_des[3])   # Desired roll angle to achieve desired velocity in x
    yaw_des = 0.0  # For simplicity, keeping desired yaw to 0

    error_yaw = yaw_des - orientation[3]
    yaw = pd_control(error_yaw, -angular_velocity[3], kp_yaw, kd_yaw)

    error_pitch = pitch_des - orientation[2]
    pitch = pd_control(error_pitch, -angular_velocity[2], kp_pitch, kd_pitch)

    error_roll = roll_des - orientation[1]
    roll = pd_control(error_roll, -angular_velocity[1], kp_roll, kd_roll)

    motor_speeds = motor_mixing(thrust_total, yaw, pitch, roll)
    println("Motor speeds: ", motor_speeds)
    set_input!(environment, motor_speeds)
end

# Position controller
function position_controller!(environment, pos_des)
    pos_is = get_state(environment)[1:3]
    v_des = pos_des .- pos_is
    control_thrust_orientation!(environment, v_des)
end

# Main controller
function controller!(environment, k)
    pos_des = [2; 0; 1.0]  # Example coordinates to fly to (x, y, z)
    position_controller!(environment, pos_des)
end

# Simulate
initialize!(quadrotor_env, :quadrotor)
simulate!(quadrotor_env, controller!; record=true)

# Visualize
vis = visualize(quadrotor_env)
render(vis)


Position: [0.0, 0.0, 0.085]
Motor speeds: [53.135415208297374, 50.851969346822514, 53.135415208297374, 50.851969346822514]
Position: [5.492671945060969e-20, -9.033178942691005e-24, 0.08647086959630063]
Motor speeds: [52.3084087549982, 50.0241252864794, 52.30878777836867, 50.02374626310893]
Position: [3.0679395166867764e-9, -3.0140163751314915e-9, 0.08845848930680593]
Motor speeds: [52.06200961511323, 49.7769948486422, 52.06330248833324, 49.77570385752146]
Position: [1.5950955790953272e-8, -1.539347050439805e-8, 0.09066378332514999]
Motor speeds: [51.96256567440899, 49.67732974080262, 51.96546429504417, 49.674440327702854]
Position: [4.810229304735268e-8, -4.556933949779328e-8, 0.09297938506597665]
Motor speeds: [51.91063219167556, 49.62595276020954, 51.916007846809364, 49.620604951472956]
Position: [1.0889499489666466e-7, -1.0239195205981813e-7, 0.09537414321095457]
Motor speeds: [51.871901908737996, 49.58858844497496, 51.88063217062539, 49.579924126170624]
Position: [1.703872894765035

┌ Info: Listening on: 127.0.0.1:8700, thread id: 1
└ @ HTTP.Servers /home/victor/.julia/packages/HTTP/sJD5V/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/victor/.julia/packages/MeshCat/0RCA3/src/visualizer.jl:64


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("Background" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}("top_color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x42, 0x61, 0x63, 0x6b, 0x67, 0x72, 0x6f, 0x75, 0x6e, 0x64], "bottom_color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x42, 0x61, 0x63, 0x6b, 0x67, 0x72, 0x6f, 0x75, 0x6e, 0x64]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Grid" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}("visible" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x70, 0x61, 0x74, 0x68, 0xa5, 0x2f, 0x47, 0x72, 0x69, 0x64]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("waypoints" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("waypoint1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "floor" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("bodies" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("rotor_3__id_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x5f, 0x33, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rotor_1__id_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x5f, 0x31, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "base_link__id_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6c, 0x69, 0x6e, 0x6b, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rotor_0__id_5" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 